## Gaussian Processes

### Neil D. Lawrence

### 12th May 2016

In [2]:
import numpy as np
import pandas as pd
import pods
import matplotlib.pyplot as plt
import mlai
import teaching_plots as plot 
from IPython.display import HTML
%matplotlib inline

ModuleNotFoundError: No module named 'mlai'

<img src="../diagrams/9780262182539-f30.jpg">
@Rasmussen:book06

### What is Machine Learning?

$$ \text{data} + \text{model} = \text{prediction}$$

-   $\text{data}$ : observations, could be actively or passively
    acquired (meta-data).

-   $\text{model}$ : assumptions, based on previous experience (other data!
    transfer learning etc), or beliefs about the regularities of
    the universe. Inductive bias.

-   $\text{prediction}$ : an action to be taken or a categorization or a
    quality score.

### Olympic Marathon Data

-   Gold medal times for Olympic Marathon since 1896.

-   Marathons before 1924 didn’t have a standardised distance.

-   Present results using pace per km.

-   In 1904 Marathon was badly organised leading to very slow times.

![image](./diagrams/Stephen_Kiprotich.jpg)
Image from Wikimedia Commons <http://bit.ly/16kMKHQ>

In [ ]:
data = pods.datasets.olympic_marathon_men()
f, ax = plt.subplots(figsize=plot.one_figsize)
ax.plot(data['X'], data['Y'], 'r.',markersize=10)
ax.set_xlabel('year', fontsize=20)
ax.set_ylabel('pace min/km', fontsize=20)
plt.savefig('../../slides/diagrams/ml/olympic_marathon.svg', transparent=True, frameon=True)


### Olympic Marathon Data

<img src="../diagrams/olympic_marathon.svg"> 

In [ ]:
plot.over_determined_system(diagrams='../../slides/diagrams/ml')

# Overdetermined System

![](diagrams/over_determined_system007.svg)

In [ ]:
pods.notebook.display_plots('over_determined_system{samp:0>3}.svg', directory='../../slides/diagrams/ml', samp=(1, 7))

# $y = mx + c$

point 1: $x = 1$, $y=3$ $$3 = m + c$$ 
point 2: $x = 3$, $y=1$ $$1 = 3m + c$$ 
point 3: $x = 2$, $y=2.5$ $$2.5 = 2m + c$$

<img src="../diagrams/Pierre-Simon_Laplace.png" align=center width=50%>

<img src="../diagrams/LaplacesDeterminismFrench.png" align=center width=50%>

<img src="../diagrams/LaplacesDeterminismEnglish.png" align=center width=50%>

<img src="../diagrams/philosophicaless00lapliala.png" align=center width=50%>

# $y = mx + c + \epsilon$

point 1: $x = 1$, $y=3$ 
$$3 = m + c + \epsilon_1$$ 

point 2: $x = 3$, $y=1$ 
$$1 = 3m + c + \epsilon_2$$ 

point 3: $x = 2$, $y=2.5$ 
$$2.5 = 2m + c + \epsilon_3$$

### The Gaussian Density
- Perhaps the most common probability density.
\begin{align*}
p(y| \mu, \sigma^2) & = \frac{1}{\sqrt{2\pi\sigma^2}}\exp\left(-\frac{(y - \mu)^2}{2\sigma^2}\right)\\
& \buildrel\triangle\over = \mathcal{N}(y|\mu, \sigma^2)
\end{align*}
- The Gaussian density.


In [ ]:
plot.gaussian_of_height(diagrams='../../slides/diagrams/ml')

### Gaussian Density
![](./diagrams/gaussian_of_height.svg)
The Gaussian PDF with $\mu=1.7$ and variance $\sigma^2=
  0.0225$. Mean shown as red line. It could represent the heights of a population of
  students.

### Gaussian Density
$$
\mathcal{N}(y|\mu, \sigma^2) =  \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{(y-\mu)^2}{2\sigma^2}\right)
$$
$\sigma^2$ is the variance of the density and $\mu$ is the mean.



### Two Important Gaussian Properties

**Sum of Gaussian**

-   Sum of Gaussian variables is also Gaussian.
    $$y_i \sim \mathcal{N}(\mu, \sigma^2)$$ 
    And the sum is distributed as
    $$\sum_{i=1}^{n} y_i \sim \mathcal{N}\left(\sum_{i=1}^n \mu_i,\sum_{i=1}^n \sigma_i^2\right)$$
    (*Aside*: As sum increases, sum of non-Gaussian, finite variance variables is
    also Gaussian [central limit theorem](https://en.wikipedia.org/wiki/Central_limit_theorem).)

### Two Important Gaussian Properties

**Scaling a Gaussian**

-   Scaling a Gaussian leads to a Gaussian.
    $$y \sim \mathcal{N}(\mu, \sigma^2)$$
    And the scaled density is distributed as
    $$w y \sim \mathcal{N}(w\mu,w^2 \sigma^2)$$

### Laplace's Idea

### A Probabilistic Process

-   Set the mean of Gaussian to be a function.
    $$p\left(y_i|x_i\right)=\frac{1}{\sqrt{2\pi\sigma^2}}\exp \left(-\frac{\left(y_i-f\left(x_i\right)\right)^{2}}{2\sigma^2}\right).$$

-   This gives us a ‘noisy function’.

-   This is known as a stochastic process.

### Height as a Function of Weight

-   In the standard Gaussian, parametized by mean and variance.

-   Make the mean a linear function of an *input*.

-   This leads to a regression model. 
    \begin{align*}
       y_i=&f\left(x_i\right)+\epsilon_i,\\
         \epsilon_i \sim &\mathcal{N}(0, \sigma^2).
     \end{align*}
        
-   Assume $y_i$ is height and $x_i$ is weight.

### Data Point Likelihood

-   Likelihood of an individual data point
    $$p\left(y_i|x_i,\sigma^2,f()\right)=\frac{1}{\sqrt{2\pi \sigma^2}}\exp \left(-\frac{\left(y_i-f(x_i)\right)^{2}}{2\sigma^2}\right).$$

-   If $f(x_i) = mx_i + c$ then parameters are gradient, $m$, offset, $c$ of the function and noise
    variance $\sigma^2$.

### Data Set Likelihood

-   If the noise, $\epsilon_i$ is sampled independently for each
    data point.

-   Each data point is independent (given $m$ and $c$).

-   For independent variables:
    $$p(\mathbf{y}) = \prod_{i=1}^n p(y_i)$$
    $$p(\mathbf{y}|\mathbf{x}, m, c) = \prod_{i=1}^n p(y_i|x_i, m, c)$$
    

### Two Simultaneous Equations

A system of two simultaneous equations with two
unknowns.

How do we deal with three simultaneous
equations with only two unknowns?
<table><tr><td>
$$\begin{aligned}
        y_1 = & mx_1 + c\\
        y_2 = & mx_2 + c
      \end{aligned}$$ 
      
$$\begin{aligned}
        y_1-y_2 = & m(x_1 - x_2)
      \end{aligned}$$  
      
$$\begin{aligned}
        \frac{y_1-y_2}{x_1 - x_2} = & m
      \end{aligned}$$ 
      
$$\begin{aligned}
        m & =\frac{y_2-y_1}{x_2 - x_1}\\
        c & = y_1 - m x_1
      \end{aligned}$$ 
      
$$\begin{aligned}
        y_1 = & mx_1 + c\\
        y_2 = & mx_2 + c\\
        y_3 = & mx_3 + c
      \end{aligned}$$</td></tr>/</table>

In [ ]:
plot.under_determined_system(diagrams='../../slides/diagrams/ml')

### Underdetermined System
- What about two unknowns and *one* observation?
    $$y_1 =  mx_1 + c$$

## Underdetermined System

In [ ]:
pods.notebook.display_plots('under_determined_system{samp:0>3}.svg', 
                            directory='../../slides/diagrams/ml', samp=(0, 10))

### Underdetermined System
- Can compute $m$ given $c$.
$$m = \frac{y_1 -c}{x}$$

### Underdetermined System

- Can compute $m$ given $c$.

Assume 
$$c \sim \mathcal{N}(0, 4),$$

### Overdetermined System

-   With two unknowns and two observations: 
    $$\begin{aligned}
          y_1 = & mx_1 + c\\
          y_2 = & mx_2 + c
        \end{aligned}$$

-   Additional observation leads to *overdetermined* system.
    $$y_3 =  mx_3 + c$$

-   This problem is solved through a noise model
    $\epsilon \sim \mathcal{N}(0,\sigma^2)$ $$\begin{aligned}
          y_1 = mx_1 + c + \epsilon_1\\
          y_2 = mx_2 + c + \epsilon_2\\
          y_3 = mx_3 + c + \epsilon_3
        \end{aligned}$$

### Noise Models

-   We aren’t modeling entire system.

-   Noise model gives mismatch between model and data.

-   Gaussian model justified by appeal to central limit theorem.

-   Other models also possible (Student-$t$ for heavy tails).

-   Maximum likelihood with Gaussian noise leads to *least squares*.

### Different Types of Uncertainty

-   The first type of uncertainty we are assuming is
    *aleatoric* uncertainty.

-   The second type of uncertainty we are assuming is
    *epistemic* uncertainty.

### Aleatoric Uncertainty

-   This is uncertainty we couldn’t know even if we wanted to. e.g. the
    result of a football match before it’s played.

-   Where a sheet of paper might land on the floor.

### Epistemic Uncertainty

-   This is uncertainty we could in principal know the answer too. We
    just haven’t observed enough yet, e.g. the result of a football
    match *after* it’s played.

-   What colour socks your lecturer is wearing.

### Prior Distribution

-   Bayesian inference requires a prior on the parameters.

-   The prior represents your belief *before* you see the data of the
    likely value of the parameters.

-   For linear regression, consider a Gaussian prior on the intercept:
    $$c \sim \mathcal{N}(0, \alpha_1)$$

### Posterior Distribution

-   Posterior distribution is found by combining the prior with
    the likelihood.

-   Posterior distribution is your belief *after* you see the data of
    the likely value of the parameters.

-   The posterior is found through **Bayes’ Rule**
    $$p(c|y) = \frac{p(y|c)p(c)}{p(y)}$$


### Bayes Update

In [ ]:
plot.bayes_update(diagrams='../../slides/diagrams/ml')

In [ ]:
pods.notebook.display_plots('dem_gaussian{stage:0>3}.svg', 
                            '../../slides/diagrams/ml', stage=(1, 3))

### Stages to Derivation of the Posterior

-   Multiply likelihood by prior

    -   they are "exponentiated quadratics", the answer is always also
        an exponentiated quadratic because
        $$\exp(a^2)\exp(b^2) = \exp(a^2 + b^2)$$

-   Complete the square to get the resulting density in the form of
    a Gaussian.

-   Recognise the mean and (co)variance of the Gaussian. This is the
    estimate of the posterior.

### Main Trick

$$p(c) = \frac{1}{\sqrt{2\pi\alpha_1}} \exp\left(-\frac{1}{2\alpha_1}c^2\right)$$
$$p(\mathbf{y}|\mathbf{x}, c, m, \sigma^2) = \frac{1}{\left(2\pi\sigma^2\right)^{\frac{n}{2}}} \exp\left(-\frac{1}{2\sigma^2}\sum_{i=1}^n(y_i - mx_i - c)^2\right)$$

$$p(c| \mathbf{y}, \mathbf{x}, m, \sigma^2) = \frac{p(\mathbf{y}|\mathbf{x}, c, m, \sigma^2)p(c)}{p(\mathbf{y}|\mathbf{x}, m, \sigma^2)}$$

$$p(c| \mathbf{y}, \mathbf{x}, m, \sigma^2) =  \frac{p(\mathbf{y}|\mathbf{x}, c, m, \sigma^2)p(c)}{\int p(\mathbf{y}|\mathbf{x}, c, m, \sigma^2)p(c) \text{d} c}$$

$$p(c| \mathbf{y}, \mathbf{x}, m, \sigma^2) \propto  p(\mathbf{y}|\mathbf{x}, c, m, \sigma^2)p(c)$$

$$\begin{aligned}
    \log p(c | \mathbf{y}, \mathbf{x}, m, \sigma^2) =&-\frac{1}{2\sigma^2} \sum_{i=1}^n(y_i-c - mx_i)^2-\frac{1}{2\alpha_1} c^2 + \text{const}\\
     = &-\frac{1}{2\sigma^2}\sum_{i=1}^n(y_i-mx_i)^2 -\left(\frac{n}{2\sigma^2} + \frac{1}{2\alpha_1}\right)c^2\\
    & + c\frac{\sum_{i=1}^n(y_i-mx_i)}{\sigma^2},
  \end{aligned}$$

complete the square of the quadratic form to obtain
$$\log p(c | \mathbf{y}, \mathbf{x}, m, \sigma^2) = -\frac{1}{2\tau^2}(c - \mu)^2 +\text{const},$$
where $\tau^2 = \left(n\sigma^{-2} +\alpha_1^{-1}\right)^{-1}$
and
$\mu = \frac{\tau^2}{\sigma^2} \sum_{i=1}^n(y_i-mx_i)$.

### The Joint Density

-   Really want to know the *joint* posterior density over the
    parameters $c$ *and* $m$.

-   Could now integrate out over $m$, but it’s easier to consider the
    multivariate case.

In [ ]:
plot.height_weight(diagrams='../../slides/diagrams/ml')

In [ ]:
plot.independent_height_weight(num_samps=8, 
                               diagrams='../../slides/diagrams/ml')

In [ ]:
pods.notebook.display_plots('independent_height_weight{stage:0>3}.svg', 
                            '../../slides/diagrams/ml', stage=(0, 7))

In [ ]:
plot.correlated_height_weight(num_samps=8, 
                              diagrams='../../slides/diagrams/ml')

In [ ]:
pods.notebook.display_plots('correlated_height_weight{stage:0>3}.svg', 
                            '../../slides/diagrams/ml', stage=(0, 7))

### Sampling a Function

**Multi-variate Gaussians**

* We will consider a Gaussian with a particular structure of covariance
    matrix.
* Generate a single sample from this 25 dimensional Gaussian distribution, $\mathbf{f}=\left[f_{1},f_{2}\dots f_{25}\right]$.

* We will plot these points against their index.


In [ ]:
# %load -s compute_kernel mlai.py
def compute_kernel(X, X2, kernel, **kwargs):
    K = np.zeros((X.shape[0], X2.shape[0]))
    for i in np.arange(X.shape[0]):
        for j in np.arange(X2.shape[0]):
            K[i, j] = kernel(X[i, :], X2[j, :], **kwargs)
        
    return K


In [ ]:
# %load -s polynomial_cov mlai.py
def polynomial_cov(x, x_prime, variance=1., degree=2., w=1., b=1.):
    "Polynomial covariance function."
    return variance*(np.dot(x, x_prime)*w + b)**degree


In [ ]:
# %load -s exponentiated_quadratic mlai.py
def exponentiated_quadratic(x, x_prime, variance=1., lengthscale=1.):
    "Exponentiated quadratic covariance function."
    squared_distance = ((x-x_prime)**2).sum()
    return variance*np.exp((-0.5*squared_distance)/lengthscale**2)        


In [ ]:
plot.two_point_sample(compute_kernel, kernel=exponentiated_quadratic, 
                      lengthscale=0.5, diagrams='../../slides/diagrams/gp')


## Gaussian Samples

In [ ]:
pods.notebook.display_plots('two_point_sample{sample:0>3}.svg', 
                            '../../slides/diagrams/gp', sample=(0,12))

<img src="../../slides/diagrams/gp/799px-Uluru_Panorama.jpg" alignment='center'>

## Prediction with Correlated Gaussians
  * Prediction of $f_2$ from $f_1$ requires *conditional density*.
  * Conditional density is *also* Gaussian.
    $$
    p(f_2|f_1) = \mathcal{N}\left(f_2|\frac{k_{1, 2}}{k_{1, 1}}f_1, k_{2, 2} - \frac{k_{1,2}^2}{k_{1,1}}\right)
    $$
    where covariance of joint density is given by
    $$
    \mathbf{K} = \begin{bmatrix} k_{1, 1} & k_{1, 2}\\ k_{2, 1} & k_{2, 2}\end{bmatrix}
    $$

## Gaussian Samples

In [ ]:
pods.notebook.display_plots('two_point_sample{sample:0>3}.svg', 
                            '../../slides/diagrams/gp', sample=(13,16))

## Prediction with Correlated Gaussians
  * Prediction of $\mathbf{f}_*$ from $\mathbf{f}$ requires multivariate *conditional density*.
  * Multivariate conditional density is *also* Gaussian.
    $$
    p(\mathbf{f}_*|\mathbf{f}) = \mathcal{N}\left(\mathbf{f}_*|\mathbf{K}_{*,\mathbf{f}}\mathbf{K}_{\mathbf{f},\mathbf{f}}^{-1}\mathbf{f},\mathbf{K}_{*,*}-\mathbf{K}_{*,\mathbf{f}} \mathbf{K}_{\mathbf{f},\mathbf{f}}^{-1}\mathbf{K}_{\mathbf{f},*}\right)
    $$

## Prediction with Correlated Gaussians
  * Prediction of $\mathbf{f}_*$ from $\mathbf{f}$ requires multivariate *conditional density*.
  * Multivariate conditional density is *also* Gaussian.
    $$
    p(\mathbf{f}_*|\mathbf{f}) = \mathcal{N}\left(\mathbf{f}_*|\boldsymbol{\mu},\boldsymbol{\Sigma}\right)
    $$
    $$
    \boldsymbol{\mu} = \mathbf{K}_{*,\mathbf{f}}\mathbf{K}_{\mathbf{f},\mathbf{f}}^{-1}\mathbf{f}     
    $$
    $$
    \boldsymbol{\Sigma} = \mathbf{K}_{*,*}-\mathbf{K}_{*,\mathbf{f}} \mathbf{K}_{\mathbf{f},\mathbf{f}}^{-1}\mathbf{K}_{\mathbf{f},*}
    $$
  * Here covariance of joint density is given by
    $$
    \mathbf{K} = \begin{bmatrix} \mathbf{K}_{\mathbf{f}, \mathbf{f}} & \mathbf{K}_{*, \mathbf{f}}\\ \mathbf{K}_{\mathbf{f}, *} & \mathbf{K}_{*, *}\end{bmatrix}
    $$
  

In [ ]:
x = np.linspace(-1, 1, 100)[:, None]

In [ ]:
fig, ax = plt.subplots(figsize=plot.one_figsize)
hcolor = [1., 0., 1.]
K = compute_kernel(x, x, exponentiated_quadratic, lengthscale=0.5)
obj = plot.matrix(K, ax=ax, type='image', bracket_style='boxes')
plt.savefig('eq_covariance.svg', transparent=True)

fig, ax = plt.subplots(figsize=plot.one_figsize)
#plot.kern_circular_sample([], K, filename='eq_covariance.gif')

## Where Did This Covariance Matrix Come From?
$$
k(\mathbf{x}, \mathbf{x}^\prime) = \alpha \exp\left(-\frac{\left\Vert \mathbf{x} - \mathbf{x}^\prime\right\Vert^2_2}{2\ell^2}\right)$$

<table cellspacing="0" cellpadding="0" border="none">
<tr><td>
<ul><li>Covariance matrix is
built using the *inputs* to
the function x.</li>

<li>For the example above it
was based on Euclidean
distance.</li>

<li>The covariance function
is also know as a kernel.</li>
</ul></td><td><img src='./diagrams/eq_covariance.svg' align='right'></td></tr>
</table>

## Where Did This Covariance Matrix Come From?
$$
k(\mathbf{x}, \mathbf{x}^\prime) = \alpha \exp\left(-\frac{\left\Vert \mathbf{x} - \mathbf{x}^\prime\right\Vert^2_2}{2\ell^2}\right)$$

<table cellspacing="0" cellpadding="0" border="none">
<tr><td>
<ul><li>Covariance matrix is
built using the *inputs* to
the function x.</li>

<li>For the example above it
was based on Euclidean
distance.</li>

<li>The covariance function
is also know as a kernel.</li>
</ul></td><td><img src='./diagrams/eq_covariance.gif' align='right'></td></tr>
</table>

In [ ]:
plot.covariance_func(x, compute_kernel, 
                     formula = r'$$k(\mathbf{x}, \mathbf{x}^\prime) = \alpha(w \mathbf{x}^\top \mathbf{x}^\prime + b)^d$$', 
                     shortname='poly', 
                     longname='Polynomial', kernel=polynomial_cov,
                     degree=4., diagrams='../../slides/diagrams/kern')


In [ ]:
HTML(open('poly_covariance.html', 'r').read())

In [ ]:
plot.covariance_func(x, compute_kernel, formula = r"""$$k(\mathbf{x}, \mathbf{x}^\prime) 
= \alpha \exp\left(-\frac{\left\Vert \mathbf{x} - \mathbf{x}^\prime\right\Vert^2_2}{2\ell^2}\right)$$""", 
                     shortname='eq', 
                     longname='Exponentiated Quadratic', kernel=exponentiated_quadratic,
                     lengthscale=0.5, diagrams='../../slides/diagrams/kern')

In [ ]:
HTML(open('eq_covariance.html', 'r').read())

In [ ]:
# %load -s mlp_cov mlai.py
def mlp_cov(x, x_prime, variance=1., w=1., b=1.):
    "MLP covariance function."
    return variance*np.arcsin((w*np.dot(x, x_prime) + b)/np.sqrt((np.dot(x, x)*w +b + 1)*(np.dot(x_prime, x_prime)*w + b + 1)))


In [ ]:
plot.covariance_func(x, compute_kernel, 
                     formula = r"""$$k(\mathbf{x}, \mathbf{x}^\prime) = 
\alpha \arcsin\left(\frac{w \mathbf{x}^\top \mathbf{x}^\prime + b}
{\sqrt{\left(w \mathbf{x}^\top \mathbf{x} + b + 1\right)
\left(w \left.\mathbf{x}^\prime\right.^\top \mathbf{x}^\prime + b + 1\right)}}\right)$$""",
                     shortname='mlp', 
                     longname='Multilayer Perceptron', kernel=mlp_cov,
                     w=10., b=10., diagrams='../../slides/diagrams/kern')

In [ ]:
HTML(open('mlp_covariance.html', 'r').read())

In [ ]:
# %load -s sinc_cov mlai.py
def sinc_cov(x, x_prime, variance=1., w=1.):
    "Sinc covariance function."
    r = np.linalg.norm(x-x_prime, 2)
    return np.sinc(np.pi*w*r)


In [ ]:
plot.covariance_func(x, compute_kernel, 
                     formula = r"""$$k(\mathbf{x}, \mathbf{x}^\prime) 
= \text{sinc}\left(w \pi  \left\Vert\mathbf{x}-\mathbf{x}^\prime\right\Vert\right)$$""",
                     shortname='sinc', 
                     longname='Sinc', kernel=sinc_cov,
                     w=0.25, diagrams='../../slides/diagrams/kern')

In [ ]:
HTML(open('sinc_covariance.html', 'r').read())

In [ ]:
# %load -s periodic_cov mlai.py
def periodic_cov(x, x_prime, variance=1., lengthscale=1., w=1.):
    "Periodic covariance function"
    r = np.linalg.norm(x-x_prime, 2)
    return variance*np.exp(-2./(lengthscale*lengthscale)*np.sin(np.pi*r*w)**2)


In [ ]:
plot.covariance_func(x, compute_kernel, 
                     formula = r"""$$k(\mathbf{x}, \mathbf{x}^\prime) 
= \alpha\exp\left(-\frac{2\sin^2\left(w \pi  \left\Vert\mathbf{x}-\mathbf{x}^\prime\right\Vert\right)}{\ell^2}\right)$$""",
                     shortname='periodic', 
                     longname='Periodic', kernel=periodic_cov,
                     w=0.25, diagrams='../../slides/diagrams/kern')

In [ ]:
HTML(open('periodic_covariance.html', 'r').read())

In [ ]:
# %load -s ratquad_cov mlai.py
def ratquad_cov(x, x_prime, variance=1., lengthscale=1., alpha=1.):
    "Rational quadratic covariance function"
    r = np.linalg.norm(x-x_prime, 2)
    return variance*(1. + r*r/(2*alpha*lengthscale*lengthscale))**-alpha


In [ ]:
plot.covariance_func(x, compute_kernel, 
                     formula = r"""$$k(\mathbf{x}, \mathbf{x}^\prime) = 
\alpha\left(1+\frac{\left\Vert\mathbf{x}-\mathbf{x}^\prime\right\Vert^2}{2\beta\ell^2}\right)^{-\beta}$$""",
                     shortname='ratquad', 
                     longname='Rational Quadratic', kernel=ratquad_cov,
                     alpha=0.25, diagrams='../../slides/diagrams/kern')

In [ ]:
HTML(open('ratquad_covariance.html', 'r').read())

In [ ]:
plot.covariance_func(x, compute_kernel, 
                     formula = r"""$$k(\mathbf{x}, \mathbf{x}^\prime) 
= \alpha \exp\left(-\frac{\left\Vert \mathbf{x} - \mathbf{x}^\prime\right\Vert_2}{\ell}\right)$$""",
                     shortname='ou', 
                     longname='Ornstein Uhlenbeck', kernel=mlai.ou_cov,
                     lengthscale=1., diagrams='../../slides/diagrams/kern')

In [ ]:
HTML(open('ou_covariance.html', 'r').read())

In [ ]:
# %load -s brownian_cov mlai.py
def brownian_cov(t, t_prime, variance=1.):
    "Brownian motion covariance function."
    if t>=0 and t_prime>=0:
        return variance*np.min([t, t_prime])
    else:
        raise ValueError("For Brownian motion covariance only positive times are valid.")


In [ ]:
t = np.linspace(0, 1, 100)[:, None]
plot.covariance_func(t, compute_kernel, 
                     formula = r"""$$k(t, t^\prime) = \alpha \min(t, t^\prime)$$""",
                     shortname='brownian', 
                     longname='Brownian Motion', kernel=brownian_cov,
                    diagrams='../../slides/diagrams/kern')

In [ ]:
HTML(open('brownian_covariance.html', 'r').read())

In [ ]:
plot.covariance_func(x, compute_kernel, 
                     formula = r"$$k(\mathbf{x}, \mathbf{x}^\prime) = \alpha\mathbf{x}^\top\mathbf{x}^\prime$$",
                     shortname='linear', 
                     longname='Linear', kernel=mlai.linear_cov, 
                    diagrams='../../slides/diagrams/kern')

In [ ]:
HTML(open('linear_covariance.html', 'r').read())

In [ ]:
plot.covariance_func(x, compute_kernel, 
                     formula = r"""$$k(\mathbf{x}, \mathbf{x}^\prime) 
= \alpha_1 k_1(\mathbf{x}, \mathbf{x}^\prime) + \alpha_2 k_2(\mathbf{x}, \mathbf{x}^\prime)$$""",
                     shortname='add', 
                     longname='Additive', kernel=mlai.add_cov, 
                     kerns=[mlai.exponentiated_quadratic, mlai.linear_cov], 
                     kern_args = [{'lengthscale': 0.1}, {'variance': 2}],
                    diagrams='../../slides/diagrams/kern')

In [ ]:
HTML(open('add_covariance.html', 'r').read())

In [ ]:
plot.covariance_func(x, compute_kernel, 
                     formula = r"""$$k(\mathbf{x}, \mathbf{x}^\prime) 
=  k_1(\mathbf{x}, \mathbf{x}^\prime) k_2(\mathbf{x}, \mathbf{x}^\prime)$$""",
                     shortname='prod', 
                     longname='Product', kernel=mlai.prod_cov, 
                     kerns=[mlai.exponentiated_quadratic, mlai.polynomial_cov], 
                     kern_args = [{'lengthscale': 0.1}, {'variance': 1, 'degree': 3, 'b': 0}],
                    diagrams ='../../slides/diagrams/kern')

In [ ]:
HTML(open('prod_covariance.html', 'r').read())

In [ ]:
plot.covariance_func(x, compute_kernel, 
                     formula = r"""$$k(\mathbf{x}, \mathbf{x}^\prime) 
= \boldsymbol{\phi}(\mathbf{x})^\top \boldsymbol{\phi(\mathbf{x}^\prime)}$$""",
                     shortname='basis', 
                     longname='Basis Function', kernel=mlai.basis_cov, 
                     basis=mlai.radial,
                    diagrams='../../slides/diagrams/kern')

In [ ]:
HTML(open('basis_covariance.html', 'r').read())

## Change point example (automatic statistician)
##  Boechner's theorem

## Prediction (noiseless)

# Gaussian Noise

### Examples:
* Add points at a time, C14 calibration

In [ ]:
import numpy as np
import pandas as pd
import pods
import matplotlib.pyplot as plt
import mlai
import teaching_plots as plot 
from IPython.display import HTML
%matplotlib inline

In [ ]:
kernel = mlai.exponentiated_quadratic
data = pods.datasets.olympic_marathon_men()
x = data['X']
y = data['Y']
num_data = x.shape[0]

data_limits = [1892, 2020]

max_basis = y.shape[0]

In [ ]:
lengthscales=(5, 100, 5)
plot.rmse_fit(x, y, param_name='lengthscale', param_range=lengthscales,  
              model=mlai.GP, kernel=kernel, variance=1., sigma2=0.04, xlim=data_limits,
              objective_ylim=[1.9,4], diagrams='../../slides/diagrams/gp')

In [ ]:
pods.notebook.display_plots('olympic_GP_exponentiated_quadratic_lengthscale{lengthscale:0>3}.svg', 
                            directory='../../slides/diagrams/gp', lengthscale=lengthscales)

In [ ]:
lengthscales=(5, 100, 5)
num_parts = 5
plot.cv_fit(x, y, param_name='lengthscale', param_range=lengthscales,  
              model=mlai.GP, kernel=kernel, variance=1., sigma2=0.04, xlim=data_limits,
              objective_ylim=[0.2,0.85], num_parts=5, diagrams='../../slides/diagrams/gp')

In [ ]:
pods.notebook.display_plots('olympic_{num_parts}'.format(num_parts=5) 
                            + 'cv{part:0>2}_GP_exponentiated_quadratic_lengthscale{lengthscale:0>3}.svg', 
                            directory='../../slides/diagrams/gp', part=(0,5),lengthscale=lengthscales)

# Non-Gaussian Noise


## Log Odds

* model the *log-odds* with the basis functions.
* [odds](http://en.wikipedia.org/wiki/Odds) are defined as the ratio of the probability of a positive outcome, to the probability of a negative outcome. 
* Probability is between zero and one, odds are:
    $$ \frac{\pi}{1-\pi} $$
* Odds are between $0$ and $\infty$. 
* Logarithm of odds maps them to $-\infty$ to $\infty$.

### Logit Link Function

* The [Logit function](http://en.wikipedia.org/wiki/Logit), $$g^{-1}(\pi_i) = \log\frac{\pi_i}{1-\pi_i}.$$ This function is known as a *link function*.

* For a standard regression we take,
    $$f(\mathbf{x}_i) = \mathbf{w}^\top \boldsymbol{\phi}(\mathbf{x}_i),$$
* For classification we perform a logistic regression. 
    $$\log \frac{\pi_i}{1-\pi_i} = \mathbf{w}^\top \boldsymbol{\phi}(\mathbf{x}_i)$$

### Inverse Link Function

We have defined the link function as taking the form $g^{-1}(\cdot)$ implying that the inverse link function is given by $g(\cdot)$. Since we have defined,
$$
g^{-1}(\pi(\mathbf{x})) = \mathbf{w}^\top \boldsymbol{\phi}(\mathbf{x})
$$
we can write $\pi$ in terms of the *inverse link* function, $g(\cdot)$ as 
$$
\pi(\mathbf{x}) = g(\mathbf{w}^\top \boldsymbol{\phi}(\mathbf{x})).
$$

In [ ]:
plot.logistic(diagrams='../../slides/diagrams/ml')

### Logistic function

* [Logistic](http://en.wikipedia.org/wiki/Logistic_function) (or sigmoid) squashes real line to between 0   & 1. Sometimes also called a 'squashing function'. 
![](../diagrams/logistic.svg)


### Prediction Function
* Can now write $\pi$ as a function of the input and the parameter vector as, $$\pi(\mathbf{x},\mathbf{w}) = \frac{1}{1+ \exp\left(-\mathbf{w}^\top \boldsymbol{\phi}(\mathbf{x})\right)}.$$

* Compute the output of a standard linear basis function composition ($\mathbf{w}^\top \boldsymbol{\phi}(\mathbf{x})$, as we did for linear regression)

* Apply the inverse link function, $g(\mathbf{w}^\top \boldsymbol{\phi}(\mathbf{x}))$. 

* Use this value in a Bernoulli distribution to form the likelihood. 

### Bernoulli Reminder

$$P(y_i|\mathbf{w}, \mathbf{x}) = \pi_i^{y_i} (1-\pi_i)^{1-y_i}$$

* Trick for switching betwen probabilities
```python
def bernoulli(y, pi):
    if y == 1:
        return pi
    else:
        return 1-pi
```

### Maximum Likelihood


* Conditional independence of data:$$P(\mathbf{y}|\mathbf{w}, \mathbf{X}) = \prod_{i=1}^n P(y_i|\mathbf{w}, \mathbf{x}_i). $$

### Log Likelihood

\begin{align*}
\log P(\mathbf{y}|\mathbf{w}, \mathbf{X}) = & \sum_{i=1}^n \log P(y_i|\mathbf{w}, \mathbf{x}_i) \\ = &\sum_{i=1}^n y_i \log \pi_i \\ & + \sum_{i=1}^n (1-y_i)\log (1-\pi_i)
\end{align*}

### Objective Function

* Probability of positive outcome for the $i$th data point $$\pi_i = g\left(\mathbf{w}^\top \boldsymbol{\phi}(\mathbf{x}_i)\right),$$ where $g(\cdot)$ is the *inverse* link function

* Objective function of the form \begin{align*}
E(\mathbf{w}) = & -  \sum_{i=1}^n y_i \log g\left(\mathbf{w}^\top \boldsymbol{\phi}(\mathbf{x}_i)\right) \\& - \sum_{i=1}^n(1-y_i)\log \left(1-g\left(\mathbf{w}^\top \boldsymbol{\phi}(\mathbf{x}_i)\right)\right).
\end{align*}

### Minimize Objective

* Grdient wrt  $\pi(\mathbf{x};\mathbf{w})$ 
\begin{align*}
\frac{\text{d}E(\mathbf{w})}{\text{d}\mathbf{w}} = & -\sum_{i=1}^n \frac{y_i}{g\left(\mathbf{w}^\top \boldsymbol{\phi}(\mathbf{x})\right)}\frac{\text{d}g(f_i)}{\text{d}f_i} \boldsymbol{\phi(\mathbf{x}_i)} \\ & +  \sum_{i=1}^n \frac{1-y_i}{1-g\left(\mathbf{w}^\top \boldsymbol{\phi}(\mathbf{x})\right)}\frac{\text{d}g(f_i)}{\text{d}f_i} \boldsymbol{\phi(\mathbf{x}_i)}
\end{align*}

### Link Function Gradient

* Also need gradient of inverse link function wrt parameters.
\begin{align*}
g(f_i) &= \frac{1}{1+\exp(-f_i)}\\
&=(1+\exp(-f_i))^{-1}
\end{align*}
and the gradient can be computed as
\begin{align*}
\frac{\text{d}g(f_i)}{\text{d} f_i} & = \exp(-f_i)(1+\exp(-f_i))^{-2}\\
& = \frac{1}{1+\exp(-f_i)} \frac{\exp(-f_i)}{1+\exp(-f_i)} \\
& = g(f_i) (1-g(f_i))
\end{align*}

### Objective Gradient

\begin{align*}
\frac{\text{d}E(\mathbf{w})}{\text{d}\mathbf{w}} = & -\sum_{i=1}^n y_i\left(1-g\left(\mathbf{w}^\top \boldsymbol{\phi}(\mathbf{x})\right)\right) \boldsymbol{\phi(\mathbf{x}_i)} \\ & + \sum_{i=1}^n (1-y_i)\left(g\left(\mathbf{w}^\top \boldsymbol{\phi}(\mathbf{x})\right)\right) \boldsymbol{\phi(\mathbf{x}_i)}.
\end{align*}

### Optimization of the Function

* Can't find a stationary point of the objective function analytically.

* Optimization has to proceed by *numerical methods*. 
    * [Newton's method](http://en.wikipedia.org/wiki/Newton%27s_method) or 
    * [gradient based optimization methods](http://en.wikipedia.org/wiki/Gradient_method) 
    
* Similarly to matrix factorization, for large data *stochastic gradient descent*  (Robbins Munroe optimization procedure) works well. 

### Poisson Distribution

* Poisson distribution is used for 'count data'. For non-negative integers, $y$, $$P(y) = \frac{\lambda^y}{y!}\exp(-y)$$

* Here $\lambda$ is a *rate* parameter that can be thought of as the number of arrivals per unit time.

* Poisson distributions can be used for disease count data. E.g. number of incidence of malaria in a district.


In [ ]:
plot.poisson(diagrams='../../slides/diagrams/ml')

### Poisson Distribution

![](../diagrams/poisson.svg)

### Poisson Regression

* In a Poisson regression make rate a function of space/time.$$\log \lambda(\mathbf{x}, t) = \mathbf{w}_x^\top \boldsymbol{\phi_x(\mathbf{x})} + \mathbf{w}_t^\top \boldsymbol{\phi}_t(t)$$

* This is known as a *log linear* or *log additive* model. 

* The link function is a logarithm.

* We can rewrite such a function as 
$$\log \lambda(\mathbf{x}, t) = f_x(\mathbf{x}) + f_t(t)$$

### Multiplicative Model

* Be careful though ... a log additive model is really multiplicative.
$$\log \lambda(\mathbf{x}, t) = f_x(\mathbf{x}) + f_t(t)$$

* Becomes $$\lambda(\mathbf{x}, t) = \exp(f_x(\mathbf{x}) + f_t(t))$$

* Which is equivalent to  $$\lambda(\mathbf{x}, t) = \exp(f_x(\mathbf{x}))\exp(f_t(t))$$

* Link functions can be deceptive in this way.

@Saul:chained16

## Optimizing Parameters

In [ ]:
%matplotlib inline
import GPy
GPy.examples.regression.multiple_optima()

In [ ]:
os.path.basename(GPy.__file__)

In [ ]:
#%load -s _contour_data ~/SheffieldML/GPy/GPy/examples/regression.py
def _contour_data(data, length_scales, log_SNRs, kernel_call=GPy.kern.RBF):
    """
    Evaluate the GP objective function for a given data set for a range of
    signal to noise ratios and a range of lengthscales.

    :data_set: A data set from the utils.datasets director.
    :length_scales: a list of length scales to explore for the contour plot.
    :log_SNRs: a list of base 10 logarithm signal to noise ratios to explore for the contour plot.
    :kernel: a kernel to use for the 'signal' portion of the data.
    """

    lls = []
    total_var = np.var(data['Y'])
    kernel = kernel_call(1, variance=1., lengthscale=1.)
    model = GPy.models.GPRegression(data['X'], data['Y'], kernel=kernel)
    for log_SNR in log_SNRs:
        SNR = 10.**log_SNR
        noise_var = total_var / (1. + SNR)
        signal_var = total_var - noise_var
        model.kern['.*variance'] = signal_var
        model.likelihood.variance = noise_var
        length_scale_lls = []

        for length_scale in length_scales:
            model['.*lengthscale'] = length_scale
            length_scale_lls.append(model.log_likelihood())

        lls.append(length_scale_lls)

    return np.array(lls)


In [ ]:
# %load -s multiple_optima ~/SheffieldML/GPy/GPy/examples/regression.py
def multiple_optima(gene_number=937, resolution=80, model_restarts=10, seed=10000, max_iters=300, optimize=True, plot=True):
    """
    Show an example of a multimodal error surface for Gaussian process
    regression. Gene 937 has bimodal behaviour where the noisy mode is
    higher.
    """

    # Contour over a range of length scales and signal/noise ratios.
    length_scales = np.linspace(0.1, 60., resolution)
    log_SNRs = np.linspace(-3., 4., resolution)

    try:import pods
    except ImportError:
        print('pods unavailable, see https://github.com/sods/ods for example datasets')
        return
    data = pods.datasets.della_gatta_TRP63_gene_expression(data_set='della_gatta',gene_number=gene_number)

    data['Y'] -= np.mean(data['Y'])

    lls = _contour_data(data, length_scales, log_SNRs, GPy.kern.RBF)
    if plot:
        plt.contour(length_scales, log_SNRs, np.exp(lls), 20, cmap=plt.cm.jet)
        ax = plt.gca()
        plt.xlabel('length scale')
        plt.ylabel('log_10 SNR')

        xlim = ax.get_xlim()
        ylim = ax.get_ylim()

    # Now run a few optimizations
    models = []
    optim_point_x = np.empty(2)
    optim_point_y = np.empty(2)
    np.random.seed(seed=seed)
    for i in range(0, model_restarts):
        # kern = GPy.kern.RBF(1, variance=np.random.exponential(1.), lengthscale=np.random.exponential(50.))
        kern = GPy.kern.RBF(1, variance=np.random.uniform(1e-3, 1), lengthscale=np.random.uniform(5, 50))

        m = GPy.models.GPRegression(data['X'], data['Y'], kernel=kern)
        m.likelihood.variance = np.random.uniform(1e-3, 1)
        optim_point_x[0] = m.rbf.lengthscale
        optim_point_y[0] = np.log10(m.rbf.variance) - np.log10(m.likelihood.variance);

        # optimize
        if optimize:
            m.optimize('scg', xtol=1e-6, ftol=1e-6, max_iters=max_iters)
        
        optim_point_x[1] = m.rbf.lengthscale
        optim_point_y[1] = np.log10(m.rbf.variance) - np.log10(m.likelihood.variance);
        
        
        if plot:
            arr = plt.Arrow(optim_point_x[0], optim_point_y[0], optim_point_x[1] - optim_point_x[0], optim_point_y[1] - optim_point_y[0])#, label=str(i), head_length=1, head_width=0.5, fc='k', ec='k')
            ax.add_patch(arr)
            m.plot()
        models.append(m)
        
    if plot:
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
    return m # (models, lls)


In [ ]:
from matplotlib import pyplot as plt
m = multiple_optima()